In [1]:
import main
import utils

In [2]:
client = main.start_db()

Found class. Skipping class creation


In [3]:
collection = utils.Collection(client, main.WV_CLASS)

In [4]:
collection.add_text_file("./data/kubernetes_concepts_overview.txt")

13

In [6]:
for wiki_title in [
    "Database",
    "Vector database",
    "Containerization (computing)",
    "Formula One",
]:
    collection.add_wiki_article(wiki_title)

In [4]:
yt_url = 'https://www.youtube.com/watch?v=iFUeV3aYynI'
collection.add_from_youtube(yt_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=iFUeV3aYynI
[youtube] iFUeV3aYynI: Downloading webpage
[youtube] iFUeV3aYynI: Downloading ios player API JSON
[youtube] iFUeV3aYynI: Downloading android player API JSON
[youtube] iFUeV3aYynI: Downloading m3u8 information
[info] iFUeV3aYynI: Downloading 1 format(s): 251
[download] Destination: temp_audio.mp3
[download] 100% of    2.09MiB in 00:00:00 at 2.97MiB/s   
Found Hello Weaviate - What Is a Vector? - downloading
[youtube] Extracting URL: https://www.youtube.com/watch?v=iFUeV3aYynI
[youtube] iFUeV3aYynI: Downloading webpage
[youtube] iFUeV3aYynI: Downloading ios player API JSON
[youtube] iFUeV3aYynI: Downloading android player API JSON
[youtube] iFUeV3aYynI: Downloading m3u8 information
[info] iFUeV3aYynI: Downloading 1 format(s): 251
[download] temp_audio.mp3 has already been downloaded
[download] 100% of    2.09MiB
Successfully Downloaded to temp_audio.mp3


/var/folders/hw/hsnpzf0926n_30_bkr09mb200000gn/T/ipykernel_66519/3451027425.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='Hello Weaviate - What Is a Vector？.mp3'>
  collection.add_from_youtube(yt_url)


5

In [5]:
print(collection.generate_summary("weaviate"))

Weaviate is a vector database that leverages vector embeddings to perform similarity-based searches and other powerful operations on different types of data. Vector embeddings are representations of meaning derived from deep learning models. Weaviate uses vectors to capture meaning in various contexts, such as text, code, images, videos, and more. It is a specialized database designed to handle vector embeddings and is used for applications like large language models, generative AI, and semantic search.


In [6]:
print(collection.suggest_topics_to_learn("weaviate"))

- Vector embeddings
- Deep learning models
- Similarity-based searches
- Vector database
- Data representation
- Semantic search
